In [1]:
using Pkg
pkg_folder = "../"
Pkg.activate(pkg_folder)
using BenchmarkTools
import Markdown; Base.showable(::MIME"text/markdown", ::Markdown.MD) = false # to make sth like """?replace"""  works for vscode backend, remember to choose Jupyter kernel
include(pkg_folder*"bnc_initialize.jl")
Threads.nthreads() # check number of threads

  Activating project at `~/Realizibility_index/bnc_julia`


24

In [3]:
N = N_generator(8,17,min_binder=4,max_binder=8)
model = Bnc(N=N)
u = get_vertices_neighbor_mat!(model)
model.direction

Start finding all vertices, it may takes a while.
Done, with 169000 vertices found and 30199 asymptotic vertices.
Start calculating nullity for each vertex, it may takes a while.
Done.
Start calculating verteices graph, It may takes a while.
Done.


1

In [354]:
q_sym = [:tE, :ttB, :tAs]
x_sym = [:E, :Bs, :As, :B, :C4, :C2, :C3]
K_sym = [:KB2, :KA2, :KB1,:Kk]
    # :E, :Bs, :As, :B, :C4, :C2, :C3

L = [  1    0   0    0    1    0     0   # tE
       0    1   0    1    1    1     1   # ttB
       0    0   1    0    0    1     1]   # tAs
    # :E, :Bs, :As, :B, :C4, :C2, :C3
N = [  1    1   0    0    -1    0    0
       0    1   1    0    0    -1    0
       0    0   1    1    0    0    -1
       0    0   0    0    -1   0    1]
model=Bnc(N=N,L=L,x_sym=x_sym,q_sym=q_sym,K_sym=K_sym)

Bnc{Int8}([1 1 … 0 0; 0 1 … -1 0; 0 0 … 0 -1; 0 0 … 0 1], [1 0 … 0 0; 0 1 … 1 1; 0 0 … 1 1], 4, 7, 3, Num[E, Bs, As, B, C4, C2, C3], Num[tE, ttB, tAs], Num[KB2, KA2, KB1, Kk], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], nothing, Dict{Vector{Int8}, Any}(), Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), 1, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.3010299956639812, 0.6989700043360189, 0.47712125471966244, 0.0, 0.0, 0.0, 0.0], false, sparse([1, 2, 3, 2, 1, 2, 2, 3, 2, 3], [1, 2, 3, 4, 5, 5, 6, 6, 7, 7], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3, 7), sparse([1, 2, 3, 2, 1, 2, 2, 3, 2, 3], [1, 2, 3, 4, 5, 5, 6, 6, 7, 7], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3, 7), [[1, 5], [2, 4, 5, 6, 7], [3, 6, 7]], sparse([1, 1, 2, 2, 3, 3, 1, 4, 2, 3, 4], [1, 2, 2, 3, 3, 4, 5, 5, 6, 7, 7], [1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1], 4, 7), sparse([1, 4, 2, 4, 5, 3, 5, 6, 2, 6  …  2, 4, 7, 2, 3, 5, 2, 3, 6, 7], [1, 1, 2, 2, 2, 3, 3, 3, 4, 4  …  5, 5, 5, 6, 6, 6, 7, 7, 7, 7], [1

In [468]:
idx1 = sample(get_vertices(model,singular=false,asymptotic=true))

6-element Vector{Int8}:
 1
 2
 3
 9
 5
 6

In [469]:
nbs_singular = get_neighbors(model,idx1;singular=true,asymptotic=true)

5-element Vector{Vector{Int8}}:
 [1, 9, 3, 9, 5, 6]
 [1, 2, 3, 9, 9, 6]
 [1, 2, 9, 9, 5, 6]
 [1, 2, 3, 9, 5, 9]
 [9, 2, 3, 9, 5, 6]

In [470]:
nbs_nonsingular = get_neighbors(model,idx1;singular=false,asymptotic=true)

6-element Vector{Vector{Int8}}:
 [1, 2, 3, 4, 5, 6]
 [1, 2, 3, 7, 5, 6]
 [1, 2, 3, 9, 8, 6]
 [1, 2, 3, 10, 5, 6]
 [1, 2, 3, 9, 5, 8]
 [8, 2, 3, 9, 5, 6]

In [471]:
# idx2 = sample(nbs_nonsingular)
idx2 = sample(nbs_singular)

6-element Vector{Int8}:
 1
 9
 3
 9
 5
 6

In [472]:
println("vtx1 is real: $(get_vertex!(model,idx1).real), nullity: $(get_nullity!(model,idx1))")
println("vtx2 is real: $(get_vertex!(model,idx2).real), nullity: $(get_nullity!(model,idx2))")

vtx1 is real: true, nullity: 0
vtx2 is real: true, nullity: 1


For two vertices that is both invertible

In [373]:
H1 = get_H!(model,idx1)
H2 = get_H!(model,idx2)
M1 = get_M!(model,idx1)
M2 = get_M!(model,idx2)
M = (M1 + M2)/2
H = inv(Array(M))|> sparse |> x->droptol!(x,1e-8)
i = findfirst(get_perm(model,idx1) .!= get_perm(model,idx2))
ei = SparseVector(model.n, [i], [1])

j1 = findfirst(get_P!(model,idx1)[i,:] .== 1)
j2 = findfirst(get_P!(model,idx2)[i,:] .== 1)
println("i is $i, j1 is $j1, j2 is $j2")

i is 6, j1 is 6, j2 is 9


In [374]:
a = find_direction_direct(model, idx1, idx2)
println("i is $i")
b = (H1[j2,:]-H2[j1,:])/2
c = H[j2,:]-H[j1,:]
d = H1[j2,:]-ei
e = ei-H2[j1,:]
display(a)
display(b)
display(c)
display(d)
display(e)
(a./b) .* (a./c) .* (a./d) .* (a./e)


LoadError: No interface found.

For vtx1 is invertible and vtx2 is not invertible

In [473]:
M1 = get_M!(model,idx1)
M2 = get_M!(model,idx2)
M = (M1 + M2)/2
H = inv(Array(M))|> sparse |> x->droptol!(x,1e-8)
i = findfirst(get_perm(model,idx1) .!= get_perm(model,idx2))
ei = SparseVector(model.n, [i], [1])
j1 = findfirst(get_P!(model,idx1)[i,:] .== 1)
j2 = findfirst(get_P!(model,idx2)[i,:] .== 1)

# calc from H:
c = H[j2,:]-H[j1,:]
display(c)
# calc from direct:
H1 = get_H!(model,idx1)
d = H1[j2,:]-ei # this formula seems to be general, but how it works for singular case? Especially both sides are singular?
display(d)
e = get_C_qK!(model,idx2)[1,:] # equality constrain
display(e)

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [2]  =  -2.0
  [4]  =  2.0

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [2]  =  -1.0
  [4]  =  1.0

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [2]  =  -1.0
  [4]  =  1.0

In [474]:
H2=get_H!(model,idx2)
-H2[j1,:] 

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [2]  =  -2.0
  [4]  =  2.0

In [476]:
-adjugate(M2)[j1,:] |> x->droptol!(x,1e-5)

10-element SparseVector{Float64, Int64} with 2 stored entries:
  [2]  =  -2.0
  [4]  =  2.0

In [460]:
function  _adj_singular_matrix(A::AbstractMatrix; atol=1e-12)::Tuple{SparseMatrixCSC,Int}
    """
    Calculate the adjoint of a singular matrix M, and return the nullity.
    """
    n, m = size(A)
    @assert n == m "A must be square"
    F = svd(Array(A))
    S = F.S
    thresh = atol * maximum(S)
    zero_ids = findall(σ -> σ ≤ thresh, S)
    nullity = length(zero_ids)
    if nullity == 1
        k = zero_ids[1]
        logσprod = sum(log, S[setdiff(1:n,[k])])
        σprod = exp(logσprod)
        sign_correction = det(F.U) * det(F.V)
        u = F.U[:, k]   # 左奇异向量
        v = F.V[:, k]   # 右奇异向量
        adj_A = (sign_correction *σprod) * (sparsevec(v) * sparsevec(u)') 
        return droptol!(adj_A,1e-10), 1  # rank-1 矩阵
        # return σprod * (v * u'), 1  # rank-1 矩阵
    else
        return spzeros(0,0), nullity
    end
end

_adj_singular_matrix (generic function with 1 method)

In [463]:
_adj_singular_matrix(-M2)[1]

10×10 SparseMatrixCSC{Float64, Int64} with 10 stored entries:
 -21.0   ⋅    ⋅    ⋅    ⋅    21.0   ⋅    ⋅    ⋅    ⋅ 
  18.0   ⋅    ⋅    ⋅    ⋅   -18.0   ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
  14.0   ⋅    ⋅    ⋅    ⋅   -14.0   ⋅    ⋅    ⋅    ⋅ 
   8.0   ⋅    ⋅    ⋅    ⋅    -8.0   ⋅    ⋅    ⋅    ⋅ 
 -28.0   ⋅    ⋅    ⋅    ⋅    28.0   ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 

In [465]:
adjugate(Array(-M2)) |> sparse |> x->droptol!(x,1e-5)

10×10 SparseMatrixCSC{Float64, Int64} with 10 stored entries:
 -21.0   ⋅    ⋅    ⋅    ⋅    21.0   ⋅    ⋅    ⋅    ⋅ 
  18.0   ⋅    ⋅    ⋅    ⋅   -18.0   ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
  14.0   ⋅    ⋅    ⋅    ⋅   -14.0   ⋅    ⋅    ⋅    ⋅ 
   8.0   ⋅    ⋅    ⋅    ⋅    -8.0   ⋅    ⋅    ⋅    ⋅ 
 -28.0   ⋅    ⋅    ⋅    ⋅    28.0   ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 
    ⋅    ⋅    ⋅    ⋅    ⋅      ⋅    ⋅    ⋅    ⋅    ⋅ 

In [285]:
function adjugate(A::AbstractMatrix)
    n, m = size(A)
    @assert n == m "adjugate: A must be square"

    # 1x1 的特殊情况（也被后文的通式包含）
    if n == 1
        return ones(eltype(A), 1, 1)
    end

    # 先尝试用行列式判断是否可逆（数值上接近 0 则认为奇异）
    d = det(A)
    # 这里用一个相对松散的零判断，适用于数值矩阵
    if !isapprox(d, zero(eltype(d)), atol=0, rtol=1e-12)
        # 可逆：adj(A) = det(A) * inv(A)
        return d * inv(A)
    end

    # 奇异：按定义计算伴随矩阵 = 转置(余子式矩阵)
    C = similar(A, Float64, n, n)  # cofactor matrix (未转置)
    # 为每个位置计算 (i,j) 的余子式的行列式并乘以 (-1)^(i+j)
    # 注意：余子式是删除第 i 行和第 j 列后得到的 (n-1)x(n-1) 矩阵
    for i in 1:n
        rows = [k for k in 1:n if k != i]
        for j in 1:n
            cols = [k for k in 1:n if k != j]
            M = A[rows, cols]                 # (n-1)x(n-1) 子矩阵（会复制）
            c = det(M)
            C[i, j] = (-1)^(i + j) * c
        end
    end

    return transpose(C)  # 伴随矩阵是余子式矩阵的转置
end

adjugate (generic function with 1 method)

In [115]:
# directly find the direction.
function find_direction_direct(Bnc::Bnc{T},from,to) where T
    row_num_for_each_cluster_CqK = [length(x)-1 for x in Bnc._valid_L_idx]
    function findcluster(n)
        for (i,val) in enumerate(row_num_for_each_cluster_CqK)
            if n <= val
                return i,n
            else
                n -= val
            end
        end
        error("n is too large")
    end

    (C1,C10) = get_C_C0_qK!(model,from)
    (C2,C20) = get_C_C0_qK!(model,to)
    
    m1, _ = size(C1)
    m2, _ = size(C2)

    function form_superplane(c1, c2; tol=1e-9)
        k = nothing
        for a in 1:length(c1)
            i, j = c1[a], c2[a]
            if (i == 0 && j != 0) || (i != 0 && j == 0)
                return false, 0
            elseif i == 0 && j == 0
                continue
            end
            k_new = i/j
            if k_new > 0
                return false, 0
            elseif k === nothing
                k = k_new
            elseif abs(k_new - k) > tol
                return false, 0
            end
        end
        return true , k
    end

    # Step 1: 枚举所有候选行对
    found_c2 = nothing
    # found_c01 = nothing
    found_idx = nothing
    count = 0
    for i in 1:m1
        for j in 1:m2
            c1 = C1[i,:]#, C01[i]
            c2 = C2[j,:]#, C02[j]
            is_candidate, k = form_superplane(c1, c2)
            if is_candidate
                # @show Array(c1), Array(c2)
                println("$i th row of C1 and $j th row of C2 form a candidate.")
                if (C10[i] < 1e-8 && C20[j] < 1e-8 ) || (abs(C10[i] / C20[j] - k) < 1e-8)
                    println("The two regimes are adjacent.")
                    cls1, nth1 = findcluster(i)
                    cls2, nth2 = findcluster(j)
                    println("The rows are from cluster $cls1 (row $nth1) and cluster $cls2 (row $nth2).")
                    count += 1
                    found_c2, found_idx = c2, (i,j)
                else
                    @show C10[i], C20[j], k, abs(C10[i] / C20[j] - k)
                    println("The two regimes are not adjacent.")
                end
            end
        end
    end
    
    if count == 0
        error("No interface found.")
    elseif count > 1
        error("More than one interface found.")
    else
        return found_c2
    end
end

find_direction_direct (generic function with 1 method)

In [41]:
smp = filter(get_vertices(model,singular=true)) do x
    get_nullity!(model,x) ==1 
end

35015-element Vector{Vector{Int8}}:
 [13, 2, 3, 4, 5, 6, 7, 8, 13]
 [17, 2, 3, 4, 5, 6, 7, 8, 17]
 [1, 2, 3, 4, 5, 6, 7, 12, 12]
 [11, 2, 3, 4, 5, 6, 7, 12, 12]
 [13, 2, 3, 4, 5, 6, 7, 12, 12]
 [16, 2, 3, 4, 5, 6, 7, 12, 12]
 [17, 2, 3, 4, 5, 6, 7, 12, 12]
 [13, 2, 3, 4, 5, 6, 7, 13, 9]
 [13, 2, 3, 4, 5, 6, 7, 13, 10]
 [1, 2, 3, 4, 5, 6, 7, 13, 13]
 [11, 2, 3, 4, 5, 6, 7, 13, 13]
 [1, 2, 3, 4, 5, 6, 7, 14, 14]
 [11, 2, 3, 4, 5, 6, 7, 14, 14]
 ⋮
 [13, 17, 11, 17, 16, 12, 14, 8, 9]
 [1, 2, 3, 17, 17, 6, 7, 8, 9]
 [11, 2, 3, 17, 17, 6, 7, 8, 9]
 [13, 2, 3, 17, 17, 6, 7, 8, 9]
 [1, 2, 11, 17, 17, 6, 7, 8, 9]
 [13, 2, 11, 17, 17, 6, 7, 8, 9]
 [1, 2, 13, 17, 17, 6, 7, 8, 9]
 [1, 2, 14, 17, 17, 6, 7, 8, 9]
 [1, 2, 15, 17, 17, 6, 7, 8, 9]
 [13, 2, 15, 17, 17, 6, 7, 8, 9]
 [1, 2, 16, 17, 17, 6, 7, 8, 9]
 [13, 2, 16, 17, 17, 6, 7, 8, 9]

In [42]:
@btime get_CqK(model, sample(smp))

  446.687 μs (4250 allocations: 504.81 KiB)


(sparse([5, 6, 7, 8, 9, 10, 11, 12, 13, 15  …  16, 18, 19, 20, 21, 25, 11, 17, 18, 19], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  16, 16, 16, 16, 16, 16, 17, 17, 17, 17], [-1.0000000000000002, 4.440892098500626e-16, -5.999999999999996, 4.440892098500626e-16, 1.9999999999999982, -1.0000000000000047, -4.440892098500626e-16, 1.4999999999999953, 0.9999999999999938, 1.7685237107266794e-16  …  1.1522294474108515e-17, -0.3333333333333329, -1.0179205084343823e-17, -0.15858764185114022, -0.12793065863840078, -0.0818377073172726, -3.0024060161137064e-16, -0.49999999999999983, -1.0008020053712367e-16, -0.1466978055192919], 27, 17), [0.0, 0.22184874961635698, 0.1695313018644432, -2.106283719505386, 0.12493873660829916, 0.3010299956639809, 0.5051499783199113, 0.3010299956639803, -0.16838332610663415, -0.16838332610663534  …  0.05612777536887843, 1.278889015899815e-16, 0.02670351461430272, -0.016969575802661778, 0.08419166305331796, -0.06632333477867272, 0.029456989678359784, -0.010855499319845203, -8.04045

In [57]:
@benchmark get_CqK(model, sample(smp),CDDLib.Library(), nothing)

BenchmarkTools.Trial: 618 samples with 1 evaluation per sample.
 Range (min … max):  6.234 ms … 12.912 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.721 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.087 ms ±  1.300 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▅█▃▁▃▅▃▄▅▅▂▇▂▁  ▁▂   ▁                                   
  ▆▆▇██████████████▅▆██▅▇██▅▇▆▆▃█▇█▄▅▅█▆▅▄▄▂▅▅▄▃▃▃▂▂▂▂▂▂▃▂▂▂ ▄
  6.23 ms        Histogram: frequency by time        11.7 ms <

 Memory estimate: 165.05 KiB, allocs estimate: 1808.

In [ ]:
function test(model, perm)
    

In [47]:
@btime _calc_C_C0_qk_nullity1(model,smp[1])[1]|> sparse

  55.224 μs (2192 allocations: 343.09 KiB)


283×17 SparseMatrixCSC{Float64, Int64} with 2728 stored entries:
⎡⢻⣿⣿⎤
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣯⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣷⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⡿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣟⎥
⎢⣿⣿⣿⎥
⎢⣿⣿⣿⎥
⎢⣹⣿⣿⎥
⎣⣿⣿⡇⎦

In [59]:
N = [1 1 -1]
model = Bnc(N=N)

Bnc{Int8}([1 1 -1], [1 0 1; 0 1 1], 1, 3, 2, Num[x₁, x₂, x₃], Num[q₁, q₂], Num[K₁], nothing, Vector{Int8}[], Dict{Vector{Int8}, Int64}(), Bool[], Int8[], nothing, Dict{Vector{Int8}, Any}(), Dict{Vector{Int8}, Tuple{SparseMatrixCSC{Float64, Int64}, Int8}}(), -1, [0.0, 0.0, 0.0], [0.3010299956639812, 0.3010299956639812, 0.0], false, sparse([1, 2, 1, 2], [1, 2, 3, 3], [1, 1, 1, 1], 2, 3), sparse([1, 2, 1, 2], [1, 2, 3, 3], [1, 1, 1, 1], 2, 3), [[1, 3], [2, 3]], sparse([1, 1, 1], [1, 2, 3], [1, 1, -1], 1, 3), sparse([1, 3, 2, 3, 1, 2, 3], [1, 1, 2, 2, 3, 3, 3], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0], 3, 3), [1, 3, 5, 6], [1, 2, 1, 2], [1, 2, 3, 3], [2, 4, 7], [3, 3, 3], [1, 2, 3], [1, 3], SparseArrays.UMFPACK.UmfpackLU{Float64, Int64}(SparseArrays.UMFPACK.Symbolic{Float64, Int64}(Ptr{Nothing} @0x0000000022fe66c0), SparseArrays.UMFPACK.Numeric{Float64, Int64}(Ptr{Nothing} @0x0000000024f061f0), 3, 3, [0, 2, 4, 7], [0, 2, 1, 2, 0, 1, 2], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0], 0, SparseArrays.UMF

In [78]:
a,b,c = get_CqK(model,[3,3])

([1.0 -1.0 0.0; 1.0 0.0 -1.0], [0.0, 0.0], BitSet([1]))

In [79]:
a

2×3 Matrix{Float64}:
 1.0  -1.0   0.0
 1.0   0.0  -1.0

In [80]:
b

2-element Vector{Float64}:
 0.0
 0.0

In [81]:
get_C_qK!(model,[3,3])

2×3 SparseMatrixCSC{Float64, Int64} with 5 stored entries:
 -1.0  1.0    ⋅ 
  1.0  1.0  -2.0

In [77]:
using Polyhedra:hrep,eliminate,MixedMatHRep,polyhedron,BlockElimination
using CDDLib
function get_CqK(Bnc::Bnc, perm; lib = CDDLib.Library(),alg=nothing)
    n = Bnc.n
    M,M0 = get_M_M0!(Bnc,perm)
    C,C0 = get_C_C0_x!(Bnc,perm)
    n_C = size(C,1)
    A = [[-M I(n)]; [-C zeros(n_C, n)] ] # As C0 is float64, no need to keep Int64 here # [-M I ; -C 0] [logx; logqK] =/< [M0, C0]
    b = [M0;C0]
    linset = BitSet(1:n) # first n row, also x index
    poly_hrep = hrep(A, b, linset)
    poly = isnothing(lib) ? polyhedron(poly_hrep) : polyhedron(poly_hrep,lib)
    # @show poly
    poly_pjt = isnothing(alg) ? eliminate(poly, linset) : eliminate(poly, linset,alg)
    # @show poly_pjt
    poly_pjt_hrep = MixedMatHRep(hrep(poly_pjt))
    (A_y, b_y, linset_y) = (poly_pjt_hrep.A, poly_pjt_hrep.b, poly_pjt_hrep.linset)
    return -A_y, b_y, linset_y
end

get_CqK (generic function with 3 methods)